# 14-813/18-813 Course Project – Option 2

### Task-I: Build and populate necessary tables

Firstly, I contruct the spark section.

In [1]:
# if you installed Spark on windows, 
# you may need findspark and need to initialize it prior to being able to use pyspark
# Also, you may need to initialize SparkContext yourself.
# Uncomment the following lines if you are using Windows!
#import findspark
#findspark.init()
#findspark.find()

import pyspark
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession

# from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("GenericAppName") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/25 22:07:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In the following, I import the MQTT training and test dataset and add a new column to identify them.

In [2]:
from pyspark import SparkFiles
from pyspark.sql.functions import lit


col_names = [
    "tcp.flags",
    "tcp.time_delta",
    "tcp.len",
    "mqtt.conack.flags",
    "mqtt.conack.flags.reserved",
    "mqtt.conack.flags.sp",
    "mqtt.conack.val",
    "mqtt.conflag.cleansess",
    "mqtt.conflag.passwd",
    "mqtt.conflag.qos",
    "mqtt.conflag.reserved",
    "mqtt.conflag.retain",
    "mqtt.conflag.uname",
    "mqtt.conflag.willflag",
    "mqtt.conflags",
    "mqtt.dupflag",
    "mqtt.hdrflags",
    "mqtt.kalive",
    "mqtt.len",
    "mqtt.msg",
    "mqtt.msgid",
    "mqtt.msgtype",
    "mqtt.proto_len",
    "mqtt.protoname",
    "mqtt.qos",
    "mqtt.retain",
    "mqtt.sub.qos",
    "mqtt.suback.qos",
    "mqtt.ver",
    "mqtt.willmsg",
    "mqtt.willmsg_len",
    "mqtt.willtopic",
    "mqtt.willtopic_len",
    "target"
]

#print (col_names.tolist())

# train_path = "/Users/ultrali/Documents/Grad/14813/project/archive/Data/FINAL_CSV/train70_augmented.csv"
# test_path = "/Users/ultrali/Documents/Grad/14813/project/archive/Data/FINAL_CSV/test30_augmented.csv"
train_path = "/Users/ultrali/Documents/Grad/14813/project/archive/Data/FINAL_CSV/train70_reduced.csv"
test_path = "/Users/ultrali/Documents/Grad/14813/project/archive/Data/FINAL_CSV/test30_reduced.csv"

df_train = spark.read.csv(train_path, header=True, inferSchema= True).toDF(*col_names)
df_test = spark.read.csv(test_path, header=True, inferSchema= True).toDF(*col_names)

In [ ]:
df_train = df_train.withColumn("dataset", lit(0))
df_test = df_test.withColumn("dataset", lit(1))

df_train.show(1, vertical=True)
df_test.show(1, vertical=True)

print(df_train.count())
print(df_test.count())

In [ ]:
df_train.printSchema()

Write the data into postgre table through pyspark jdbc API and verify the result with read command.

In [ ]:
db_properties={}
#update your db username
db_properties['username']="14813"
#update your db password
db_properties['password']="bigdata"
#make sure you got the right port number here
db_properties['url']= "jdbc:postgresql://localhost:5432/postgres"
#make sure you had the Postgres JAR file in the right location
db_properties['driver']="org.postgresql.Driver"
db_properties['table']= "mqtt"


df_train.write.format("jdbc")\
.mode("overwrite")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", db_properties['password'])\
.option("Driver", db_properties['driver'])\
.save()

df_test.write.format("jdbc")\
.mode("append")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", db_properties['password'])\
.option("Driver", db_properties['driver'])\
.save()


In [ ]:

sqlContext = SQLContext(spark.sparkContext)
df_read = sqlContext.read.format("jdbc")\
    .option("url", db_properties['url'])\
    .option("dbtable", db_properties['table'])\
    .option("user", db_properties['username'])\
    .option("password", db_properties['password'])\
    .option("Driver", db_properties['driver'])\
    .load()

df_read.show(1, vertical=True)